In [ ]:
import pandas as pd
import numpy as np

In [ ]:
DIR = '../udataset/acs/'

income_data = 'INCOME AND BENEFITS.csv'
income_df = pd.read_csv(DIR + income_data)
poverty_data = 'PERCENT POVERTY LINE.csv'
poverty_df = pd.read_csv(DIR + poverty_data)

print(poverty_df.columns)

industry_data = 'INDUSTRY.csv'
industry_df = pd.read_csv(DIR + industry_data)
health_data = 'HEALTH INSURANCE COVERAGE.csv'
health_df = pd.read_csv(DIR + health_data)
unemployment_data = 'unemployment.csv'
unemployment_df = pd.read_csv(DIR + unemployment_data)
worker_class_data = 'CLASS OF WORKER.csv'
worker_class_df = pd.read_csv(DIR + worker_class_data)

def strip_columns(df):
    df.columns = df.columns.str.strip()
    df['state'] = df['state'].str.strip()

    df['state'] = df['state'].str.lower()

    if 'label' in df.columns:
        df['label'] = df['label'].str.strip()
        df['label'] = df['label'].str.lower()

    return df

In [ ]:
unemployment_df = strip_columns(unemployment_df)
poverty_df = strip_columns(poverty_df)
industry_df = strip_columns(industry_df)
income_df = strip_columns(income_df)
worker_class_df = strip_columns(worker_class_df)

print(unemployment_df.columns)
unemployment_df = unemployment_df.drop(['estimate'], axis=1)

keep_cols = ['state', 'percent', 'year']
keep_cols_income = ['state', 'estimate', 'year']

filtered_poverty_df = poverty_df[poverty_df['label'] == 'all people'][keep_cols]
filtered_poverty_df = filtered_poverty_df.reset_index()
filtered_income_df = income_df[income_df['label'] == 'median household income (dollars)'][keep_cols_income]
unemployment_df = unemployment_df.reset_index()
income_df = income_df.reset_index()

filtered_poverty_df.dtypes

In [ ]:
filtered_income_df

In [ ]:
unemployment_df = pd.merge(unemployment_df, filtered_poverty_df, on=['state', 'year'])
unemployment_df = unemployment_df.drop(columns=["index_x", "index_y"])

In [ ]:
unemployment_df = unemployment_df.rename(columns={'percent':'below_poverty_line_percent'})

In [ ]:
unemployment_df = pd.merge(unemployment_df, filtered_income_df, on=['state', 'year'])

In [ ]:
unemployment_df = unemployment_df.rename(columns={'estimate':'median_household_income'})

In [ ]:
def get_label_function(desired_attr, df, desired_col):

    def get_attr_column(row):
        row = df[(df['label'] == desired_attr) & (df['year'] == row['year']) & (df['state'] == row['state'])][desired_col]
        row = row.reset_index()
        row = np.array(row)
        to_return = row[0][1]
        
        return to_return
    
    return get_attr_column

In [ ]:
attr = 'mean cash public assistance income (dollars)'
unemployment_df[attr] = unemployment_df.apply(get_label_function(attr, income_df, 'estimate'), axis=1)
unemployment_df

In [ ]:
desired_attributes = industry_df['label'].unique()

for i in range(1, len(desired_attributes)):
    attr = desired_attributes[i]
    unemployment_df[attr] = unemployment_df.apply(get_label_function(attr, industry_df, 'percent'), axis=1)

In [ ]:
desired_attributes = worker_class_df['label'].unique()
print(desired_attributes)

for i in range(1, len(desired_attributes)):
    attr = desired_attributes[i]
    unemployment_df[attr] = unemployment_df.apply(get_label_function(attr, worker_class_df, 'percent'), axis=1)

In [ ]:
unemployment_df = unemployment_df.drop(columns = ['labour_force_percent'])

In [ ]:
print(unemployment_df.dtypes)
for col in unemployment_df.columns[2:]:
    unemployment_df[col] = (unemployment_df[col] - unemployment_df[col].min())/(unemployment_df[col].max() - unemployment_df[col].min())

In [ ]:
for col in unemployment_df.columns[2:5]:
    unemployment_df[col] = unemployment_df[col]/3

for col in unemployment_df.columns[5:18]:
    unemployment_df[col] = unemployment_df[col]/13

for col in unemployment_df.columns[18:22]:
    unemployment_df[col] = unemployment_df[col]/4

In [ ]:
NEWNAME = 'state_data.csv'
unemployment_df.to_csv('../udataset/' + NEWNAME, index=False)